In [1]:
import datasets
import transformers
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [2]:
lang = ['tso', 'kin', 'swa'] # only languages related to rw

In [3]:
af_ds_list = []

for l in lang:
    try:
        ds = datasets.load_dataset('shmuhammad/AfriSenti-twitter-sentiment', l)
        af_ds_list.append(ds)
    except:
        print(f'error at {l}')

In [4]:
for i in range(len(af_ds_list)):
    print(f'{lang[i]}')
    print(af_ds_list[i]['train'][0])

tso
{'tweet': '@user Loku u navela Ku tissunga, tissungue🕳️', 'label': 2}
kin
{'tweet': '@user @user @user @user @user @user @user Hhhhhh ntabyihogoza, ubu x abo yishe bangana ik', 'label': 2}
swa
{'tweet': 'Kwani tanesco wanakataga umeme makusudinadhani kuna changamoto behind zinatakiwa zitatuliwe na sio kutoa matamko', 'label': 2}


In [5]:
count = 0

for ds in af_ds_list:
    for split in ['train', 'test', 'validation']:
        if split in ds:
            count += ds[split].num_rows

count

9427

In [6]:
full_text = []
    
for dataset in af_ds_list:
    for split in ['train', 'test', 'validation']:
        if split in dataset:
            texts = dataset[split]['tweet']
            full_text.extend(texts)

len(full_text)

9427

In [7]:
import pandas as pd

kirnews_train = pd.read_csv('news-datasets/KIRNEWS/cleaned/train.csv')
kirnews_test = pd.read_csv('news-datasets/KIRNEWS/cleaned/test.csv')

kinnews_train = pd.read_csv('news-datasets/KINNEWS/cleaned/train.csv')
kinnews_test = pd.read_csv('news-datasets/KINNEWS/cleaned/test.csv')

In [8]:
len(kirnews_train), len(kirnews_test), len(kinnews_train), len(kinnews_test)

(3689, 923, 17014, 4254)

In [9]:
news = pd.concat([kirnews_train, kirnews_test, kinnews_train, kinnews_test])
news

,label,title,content
0,1,denise nkurunziza yoba yajanwe kenya kuvurwa c...,umutambukanyi w’umukuru w’igihugu c’uburundi a...
1,1,sentare ijejwe kubahiriza ibwirizwa nshingiro ...,nkuko umukuru w’umugambwe cnl agathon rwasa yi...
2,1,sentare yiyungurizo ntahangwa yagumije munyoro...,abo bamenyeshamakuru bakaba bakorera ikinyamak...
3,1,ibisikanya mabanga y’umwibutsamana mukuru gisi...,kuri w’imana i genekerezo ntwarante umwaka nih...
4,2,umukinyi w’umurundi saido berahino yirukanwe s...,umukinyi w’umurundi akongera akaba akinira gih...
...,...,...,...
4249,2,volleyball apr bagabo rra bagore zegukanye car...,nyuma kubura igikombe shampiyona umwaka ushize...
4250,1,gisagara imbwa zizerera zikomeje kwibasira ama...,ibi abaturage babitangaje nyuma yaho mbwa zima...
4251,3,rdb yahembye batanu bagaragaje gutanga service...,ikigo k’igihugu gishinzwe iterambere rdb cyash...
4252,2,reba ibyamamare bizwi cyane byanze ’kurushinga...,gushyingirwa ntabwo garanti y’ibyishimo bamwe ...


In [10]:
for index, row in news.iterrows():
    title = row['title']
    content = row['content']
    full_text.append(title)
    full_text.append(content)

len(full_text)

61187

In [11]:
full_text

['@user Loku u navela Ku tissunga, tissungue🕳️',
 'Aha... Auxeni ti nguavani',
 'Hahahahaha ih, tipo arrombaram o nhoster, 😂😂👏 https://t.co/OTTuJlkK3l',
 'Hiiiii ni livrarih !!',
 '@user 😱 oh hosi yanga 🙆',
 '@user Eeeeh a nsati lwei a banana maj...',
 'Mu nga ni landzi... Na loya',
 '@user A ta ti nhela😂',
 'Wa ni Danissa 🙄 https://t.co/1pXjbA0QSI',
 'U gama u Kuma 2 valores https://t.co/TfQYeTa4kI',
 'Ihhhhhhhhhhh... A chirami ka lomuu!!!!',
 '@user então ungah ni trapalhari wena pah...',
 'Hahahah swa nsila só 😂😂😂😂 https://t.co/NILtGhy1pu',
 'Famba ni ma Range ya wena 😂😂😂😂',
 'A xilungu xakarata. 🤦🏾',
 '@user Famba ti gumula hi parede',
 'U bihile demais, lokhu ulangunta hansi a xikwembo swo kubha I mazwidji. 🤣🤣🤣',
 'Yuh... Mamã... A ni ku lavi mina🕺🏿',
 'I tinholuene a xwi Dari https://t.co/hAYe2sMVrB',
 '@user Wa hlanha lweyi😂😂',
 '@user KI está a criar um Vuku Vuku grande sabe',
 'Messi Wa Nhine 😂',
 '@user 🙆 hosi yanga 😹😹😹',
 '@user Uta ni dhlaya',
 'Xitomb* xa wuako https://t.c

In [12]:
with open('text_dump.txt', 'w') as f:
    for text in full_text:
        f.write(str(text)+'\n')

In [14]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
    input='text_dump.txt',
    model_prefix='sp-model/afrisenti',
    vocab_size=106828,
    model_type='unigram',
    character_coverage=0.9995,
    user_defined_symbols=['<pad>', '<mask>','<sep>','<cls>', '<s>', '</s>']
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: text_dump.txt
  input_format: 
  model_prefix: sp-model/afrisenti
  model_type: UNIGRAM
  vocab_size: 106828
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: <pad>
  user_defined_symbols: <mask>
  user_defined_symbols: <sep>
  user_defined_symbols: <cls>
  user_defined_symbols: <s>
  user_defined_symbols: </s>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all

In [15]:
sp = spm.SentencePieceProcessor()
sp.load('sp-model/afrisenti.model')

def analyze_coverage(text):
    tokens = sp.encode(text, out_type=str)
    unk_count = tokens.count('[UNK]')
    return 1 - (unk_count / len(tokens))

with open('text_dump.txt', 'r') as f:
    texts = f.readlines()

coverage = sum(analyze_coverage(text) for text in texts) / len(texts)
print(f"Token coverage: {coverage:.2%}")

Token coverage: 100.00%


In [ ]:
# sentencepiece doesnt work with electra - kill me

In [16]:
from tokenizers import ByteLevelBPETokenizer
from transformers import XLMRobertaTokenizerFast
from datasets import load_dataset
import os

def get_training_corpus(dataset, batch_size=1000):
    for start_idx in range(0, len(dataset), batch_size):
        samples = dataset[start_idx : start_idx + batch_size]
        yield samples["text"]

dataset = load_dataset("text", data_files={"train": "text_dump.txt"})["train"]

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 61187
})

In [ ]:
# trying with sentencepiece from transformers

In [17]:
from tokenizers import SentencePieceBPETokenizer

tokenizer = SentencePieceBPETokenizer()

tokenizer.train_from_iterator(
    get_training_corpus(dataset), 
    vocab_size=250000, 
    min_frequency=2,
    show_progress=True,
)

In [18]:
tokenizer

Tokenizer(vocabulary_size=229491, model=SentencePieceBPE, unk_token=<unk>, replacement=▁, add_prefix_space=True, dropout=None)

In [19]:
xlmr_tokenizer = XLMRobertaTokenizerFast(
    tokenizer_object=tokenizer
)

In [20]:
xlmr_tokenizer

XLMRobertaTokenizerFast(name_or_path='', vocab_size=229491, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	229491: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	229492: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	229493: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	229494: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [21]:
os.makedirs('xlmr-tokenizer', exist_ok=True)
xlmr_tokenizer.save_pretrained('xlmr-tokenizer')

('xlmr-tokenizer/tokenizer_config.json',
 'xlmr-tokenizer/special_tokens_map.json',
 'xlmr-tokenizer/tokenizer.json')

In [22]:
from transformers import ElectraTokenizerFast

electra_tokenizer = ElectraTokenizerFast(
    tokenizer_object=tokenizer
)

electra_tokenizer

ElectraTokenizerFast(name_or_path='', vocab_size=229491, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	229491: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	229492: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	229493: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	229494: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	229495: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=

In [23]:
electra_tokenizer.save_pretrained('electra-tokenizer')

('electra-tokenizer/tokenizer_config.json',
 'electra-tokenizer/special_tokens_map.json',
 'electra-tokenizer/vocab.json',
 'electra-tokenizer/merges.txt',
 'electra-tokenizer/added_tokens.json',
 'electra-tokenizer/tokenizer.json')